# KI Assistiertes Lehren

# Tutorial: Inhalte erstellen per KI

In diesem Tutorial werden folgende Inhalte vorgestellt:

- Erstellung von Inhalten per einfachem Prompt
- Erstellung von Inhalten per One-Shot Prompting
- Erstellung von Inhalten aus anderen Quellen per RAG (Retrieval Augmented Generation)

Notwendige Kenntnisse:

- Python

Mein Kontakt:

- Niklas Beuter (niklas.beuter@th-luebeck.de)

Als erstes installieren wir benötigte Pakete:

In [ ]:
!pip install langchain langchain-community langchain-core langchain-huggingface huggingface-hub text_generation

Nun stellen wir eine Verbindung zu einem LLM her. Hier kann jedes gehostete LLM verwendet werden, z.B. auch Chat-GPT. Wir verwenden Modelle von unserem TH Lübeck Server.

In [ ]:
# Konstanten
HOST = "https://chat-{model}.llm.mylab.th-luebeck.dev"
MODELLE = ["slim", "default", "large"]
modell_name = MODELLE[2]

# Globale Variable
inference_api_url = HOST.format(model=modell_name)
print(inference_api_url)

import os
os.environ["OPENAI_API_KEY"] = ""
HF_TOKEN = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

Langchain ist eine sehr nützliche Bibliothek zum Aufsetzen von KI-Pipelines. Mehr dazu findet Ihr auch hier: 

* [Tutorial zu Langchain von THL Kollegen Nane Kratzke, Keno Teppris](https://git.mylab.th-luebeck.de/gpu/tutorials/-/blob/c0cb6721bbd7eb4e092e1f30c6d7af8ea6413891/32-langchain.ipynb)
* [Tutorial zu Langchain+RAG von THL Kollegen Nane Kratzke](https://git.mylab.th-luebeck.de/gpu/tutorials/-/blob/c0cb6721bbd7eb4e092e1f30c6d7af8ea6413891/33-rag.ipynb)
* [Langchain](https://python.langchain.com/v0.1/docs/get_started/introduction/)

In [ ]:
from langchain.llms import HuggingFaceTextGenInference
from langchain_huggingface import HuggingFaceEndpoint
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Erstellen Sie das LLM-Objekt
llm = HuggingFaceEndpoint(
    # cache=None,  # Optional: Cache verwenden oder nicht
    verbose=True,  # Ob ausführliche Ausgaben angezeigt werden sollen
    callbacks=[StreamingStdOutCallbackHandler()],  # Callbacks, wir verwenden den fürs Streaming: Raus nehmen wenn nicht streaming genutzt werden soll
    max_new_tokens=1024,  # Die maximale Anzahl an Tokens, die generiert werden sollen
    # top_k=2,  # Die Anzahl der Top-K Tokens, die beim Generieren berücksichtigt werden sollen
    top_p=0.95,  # Die kumulative Wahrscheinlichkeitsschwelle beim Generieren
    typical_p=0.95,  # Die typische Wahrscheinlichkeitsschwelle beim Generieren
    temperature=0.1,  # Die "Temperatur" beim Generieren, gibt an wie
    # repetition_penalty=None,  # Wiederholungsstrafe beim Generieren
    # truncate=None,  # Schneidet die Eingabe-Tokens auf die gegebene Größe
    # stop_sequences=None,  # Eine Liste von Stop-Sequenzen beim Generieren
    #inference_server_url=inference_api_url,  # URL des Inferenzservers
    endpoint_url=inference_api_url,
    timeout=10,  # Timeout in Sekunden für die Verbindung zum Inferenzserver
    streaming=True,  # Ob die Antwort gestreamt werden soll,
    huggingfacehub_api_token=HF_TOKEN
)

In [ ]:
from langchain.prompts import PromptTemplate

SYSTEM_PROMPT = "Du bist ein hilfreicher Assistent und erstellst Materialien für Deep Learning." # ändere hier den System Prompt für deinen Bot.

PROMPT_TEMPLATE = PromptTemplate.from_template(
    SYSTEM_PROMPT + "\n\n{history}\nUSER:{input}\nASSISTANT:"
)

print(PROMPT_TEMPLATE.template)

Wir erstellen uns zusätzlich einen Speicher für unsere Konversation, um auch auf vorherige Punkte der Konversation eingehen zu können.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# folgende Funktionen nur zu besseren Darstellung
#def make_bold(string):
#    """Makes a String bold."""
#    return "\033[1m" +  string + "\033[0m"

#def print_section(title, content):
#    """Prints a section with a bold title and the content."""
#    print(f"{make_bold(title)}\n{content}\n")

# Erstellen eines Memory Buffers
memory = ConversationBufferMemory(
    human_prefix="USER",
    ai_prefix="ASSISTANT"
)

memory.clear()  # Bisherigen Chatverlauf löschen

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=PROMPT_TEMPLATE
)

## Erstellung von Folien über Latex

Über einfaches Prompting lässt sich auch Latex-Code ausgeben.

In [ ]:
# Latex slide example
memory.clear()

last_answer = conversation.invoke("Erstelle mir Folien zu den verschiedenen Bereichen des Deep Learnings. \
    Die Ausgabe soll in Latex Beamer Slides als Code erfolgen. Keine Überschriften oder Latex-Code Blöcke")

# Erstellung von Skripten aus Vorlesungsfolien

Um aus bestehenden Materialien neue Materialien zu erstellen, muss die KI auf die Inhalte zugreifen können. Hierfür müssen weitere Python-Pakete installiert werden, welche diese Dokumente importierbar machen. 

* PDF Dokumente lassen sich über viele PDF Pakete lesen- Hier verwenden wir `PyMuPDF`
* Word Dateien lassen sich über `docx` einlesen
* Powerpoint lassen sich über `pptx` einlesen

Nachdem Import der Dokumente können diese entweder direkt in den Prompt mit aufgenommen werden oder falls eine schnelle Suche auf den Daten ermöglicht werden soll, indiziert werden. 

Man kann auch direkt Powerpoint Folien erstellen. Ein Beispiel dazu findet man hier: https://github.com/leonid20000/odin-slides

In [ ]:
!pip install PyMuPDF chromadb langchain huggingface-hub --upgrade

Wer mehr Details über den Import von PDF Dateien haben möchte, kann hier nachlesen: [Langchain PDF Loader](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/)

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# der splitter zerteilt den Text in Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800, # größe eines Chunks 
    chunk_overlap  = 20,
    length_function = len
)
loader = PyMuPDFLoader("02_dl_supervisedlearning.pdf", extract_images=False)

docs = loader.load_and_split(text_splitter)
# Ausgabe der Seite 2 des PDF's
docs[1]

In [ ]:
## generate skript by adding text per page
#result_text = []
memory.clear()
for counter, i in enumerate(docs):
    query = "Schreibe ein Skript zu dem folgenden Inhalt. \
    Erstelle einen ausführlichen Text basierend auf den Informationen der folgenden Folie. \
    Ergänze Inhalte, wenn die Information zu wenig ist. \
    Wenn die Information auf der Folie zu dünn ist, dann lasse die Folie aus. \
    Ergänze den Text zu deinem bisherigen Text. \.:\n" + i.page_content
    last_answer = conversation.invoke(query)
    #result_text.append(last_answer)
    if counter >= 7:
        print(last_answer)
        break

## Erstellung von Prüfungsfragen

Sehr hilfreich ist die KI bei der Erstellung von Prüfungsfragen.

In [ ]:
memory.clear()

last_answer = conversation.invoke("Erstelle drei Multiple-Choice Fragen zur Lernrate. Keine Überschriften. \
Die Ausgabe als Code und auf Deutsch. \
Halte dich an folgende Struktur: Frage A) Antwort B) Antwort C) Antwort D) Antwort ANSWER: A B C or D EXPLANATION: ")

#last_answer = conversation.invoke("Erstelle drei weitere Fragen im gleichen Ausgabeformat zu Regularisierung.")

Es gibt verschiedene Möglichkeiten, die Prompts zu schreiben, um bestimmte Ausgabeformate zu erzwingen. Einige Beispiele werden z.B. [hier für H5P](https://h5p.org/ai) erklärt.

## Erstellung von XML basierten Moodle Imports

Noch besser funktioniert die Ausgabe, wenn man direkt ein Template (One-Shot Prompting) verwendet. Dies wird für alle Fragen für Moodle empfohlen, da der Import als XML die reichhaltigste Variante des Inputs darstellt und viele Details mit aufgenommen werden können.

In [ ]:
memory.clear()

last_answer = conversation.invoke("Erstelle Aussagen zu Regularisierungstechniken, die entweder wahr oder falsch sind. \
Schreibe jede Aussage an die entsprechende Stelle <shorttext>Aussage 1,2,3 oder 4</shorttext> und \
ersetze das Wort Aussage 1,2,3 oder 4. Adaptiere die auf jede Aussage folgende Zeilen <weight-of-col>0</weight-of-col> \
und schreibe dort in die erste Zeile eine 1 und in die darauffolgende Spalte eine 0, bei einer wahren Aussage. \
Bei einer falschen Aussage vertauscht Du die 1 und 0. Für die richtige Antwort jeweils unter Feedback ein. \
Nutze folgendes Template und fülle dies aus:\
<?xml version=\"1.0\" encoding=\"UTF-8\"?> \
<quiz> \
  <question type=\"matrix\"> \
    <name> \
      <text>Batch Normalization 1</text> \
    </name> \
    <questiontext format=\"html\"> \
      <text><![CDATA[<p dir=\"ltr\" style=\"text-align: left;\">Taskdescription</p>]]></text> \
    </questiontext> \
    <generalfeedback format=\"html\"> \
      <text></text> \
    </generalfeedback> \
    <defaultgrade>4</defaultgrade> \
    <penalty>0</penalty> \
    <hidden>0</hidden> \
    <idnumber></idnumber> \
    <use_dnd_ui>0</use_dnd_ui> \
    <row> \
        <shorttext>Aussage 1</shorttext> \
        <feedback format=\"html\"> \
      <text></text> \
        </feedback> \
    </row> \
    <weights-of-row> \
    <weight-of-col>0</weight-of-col> \
    <weight-of-col>1</weight-of-col> \
    </weights-of-row> \
    <row> \
        <shorttext>Aussage 2</shorttext> \
        <feedback format=\"html\"> \
      <text></text> \
        </feedback> \
    </row> \
    <weights-of-row> \
    <weight-of-col>1</weight-of-col> \
    <weight-of-col>0</weight-of-col> \
    </weights-of-row> \
    <row> \
        <shorttext>Aussage 3</shorttext> \
        <feedback format=\"html\"> \
      <text></text> \
        </feedback> \
    </row> \
    <weights-of-row> \
    <weight-of-col>1</weight-of-col> \
    <weight-of-col>0</weight-of-col> \
    </weights-of-row> \
    <row> \
        <shorttext>Aussage 4</shorttext> \
        <feedback format=\"html\"> \
      <text></text> \
        </feedback> \
    </row> \
    <weights-of-row> \
    <weight-of-col>0</weight-of-col> \
    <weight-of-col>1</weight-of-col> \
    </weights-of-row> \
    <col> \
        <shorttext>Wahr</shorttext> \
        <description format=\"html\"> \
      <text></text> \
        </description> \
    </col> \
    <col> \
        <shorttext>Falsch</shorttext> \
        <description format=\"html\"> \
      <text></text> \
        </description> \
    </col> \
    <grademethod>all</grademethod> \
    <shuffleanswers>1</shuffleanswers> \
    <multiple>0</multiple> \
    <renderer>matrix</renderer> \
  </question> \
</quiz>")

Man kann auf diese Weise viele verschiedene Fragen für Moodle erstellen. Als nächstes ein Beispiel für eine [Drop-Down Frage](https://docs.moodle.org/404/de/Fragetyp_L%C3%BCckentextauswahl):

In [ ]:
memory.clear()

last_answer = conversation.invoke("Erstelle eine Aussage zur Lernrate. Wir wollen eine Dropdown-Auswahlmöglichkeit erzeugen. \
Entferne dazu den wichtigen Teil der Aussage und schreibe den unwichtigen Teil an die Stelle STATEMENT. \
Ersetze in dem Template die Stellen FILL 1,2,3 mit zwei falschen vervollständigungen der Aussage und einmal mit der richigen Aussage. \
Vor jede falsche Vervollständigung kommt eine Tilde \"~\" und vor die richtige Vervollständigung eine Tilde und ein Gleichheitszeichen \"~=\" \
Die Ersetzung soll hinter dem Wort \"MULTICHOICE:\" beginnen. \
Schreibe jeweils direkt hinter eine Vervollständigung an die Stelle FEEDBACK 1,2,3 in dem Template eine Aussage, \
warum die Ergänzung richtig oder falsch ist. Mache zwischen Vervollständigung und Feedback eine Raute \"#\". \
<?xml version=\"1.0\" encoding=\"UTF-8\"?> \
<quiz> \
  <question type=\"cloze\"> \
    <name> \
      <text>TITLE</text> \
    </name> \
    <questiontext format=\"html\"> \
      <text><![CDATA[<p dir=\"ltr\" style=\"text-align: left;\">STATEMENT {1:MULTICHOICE: FILL 1#FEEDBACK 1~FILL 2#FEEDBACK 2~=FILL 3#FEEDBACK 3}<br></p><p></p>]]></text> \
    </questiontext> \
    <generalfeedback format=\"html\"> \
      <text></text> \
    </generalfeedback> \
    <penalty>0.3333333</penalty> \
    <hidden>0</hidden> \
    <idnumber></idnumber> \
  </question> \
</quiz>")

Hier ein Beispiel für eine [Drag & Drop Frage](https://docs.moodle.org/404/de/Fragetyp_Drag-and-Drop_auf_Text).

In [ ]:
memory.clear()

last_answer = conversation.invoke("Erstelle einen Text zur Erklärung von linearer Regression. Lasse dabei Schluesselwoerter aus und ersetze diese\
durch aufsteigende Zahlen in doppelten eckigen Klammern \"[[1]]\". Schreibe diesen Text in folgendes Template an die Stelle TEXT. \
Schreibe die Schluesselwoerter in der richtigen Reihenfolge ihres Auftretens jeweils in ein dragbox xml-statement. Die Zahl der Group soll immer 1 sein. \
Füge am Ende noch drei weitere Schluesselworter, welche nicht exakt in den Text passen, in gleicher Weise an. \
Keine Zahl darf doppelt auftreten. \
<?xml version=\"1.0\" encoding=\"UTF-8\"?>\
<quiz>\
  <question type=\"ddwtos\">\
    <name>\
      <text>TITLE</text>\
    </name>\
    <questiontext format=\"html\">\
      <text>TEXT</text>\
    </questiontext>\
    <generalfeedback format=\"html\">\
      <text></text> \
    </generalfeedback> \
    <defaultgrade>3</defaultgrade> \
    <penalty>1</penalty> \
    <hidden>0</hidden> \
    <idnumber></idnumber> \
    <shuffleanswers>1</shuffleanswers> \
    <correctfeedback format=\"html\"> \
      <text>Die Antwort ist richtig.</text> \
    </correctfeedback> \
    <partiallycorrectfeedback format=\"html\"> \
      <text>Die Antwort ist teilweise richtig.</text> \
    </partiallycorrectfeedback> \
    <incorrectfeedback format=\"html\"> \
      <text>Die Antwort ist falsch.</text> \
    </incorrectfeedback> \
    <shownumcorrect/> \
    <dragbox> \
      <text>FILLTEXT</text> \
      <group>1</group> \
    </dragbox> \
  </question> \
</quiz>")

## Übungsaufgaben

In [ ]:
memory.clear()

last_answer = conversation.run("Erstelle drei Übungsaufgaben im Bereich Physik zum Thema Lichtbrechung. Die Aufgaben sollen von leicht nach schwer gegliedert sein.")

memory.clear()

last_answer = conversation.run("Erstelle drei Übungsaufgaben für ein Jupyter Notebook. Die Beschreibung soll in Markdown und etwaiger Code in Python sein. Zeige die Ergebnisse. Gib keine extra Überschriften mit aus.")

In [ ]:
memory.clear()

last_answer = conversation.run("Erstelle eine Python-Übungsaufgabe zum Thema Intersection over Union. \
    Lasse dafür entscheidenen Code aus. Die Stelle zum Bearbeiten soll klar mit \"Ihr Code hier\" gekennzeichnet sein. \
    Ausgabe als reiner Code ohne extra Überschriften.")

## AI Tutor

Wir können die Dokumente aber nicht nur zur Erstellung von neuen Inhalten verwenden. Wir können diese auch für einen Chatbot verwenden. Dazu fügen wir alle Inhalte in eine leicht durchsuchbare Datenbank und stellen diese unserer KI zur Verfügung.

In [ ]:
!pip install langchain_openai

In [ ]:
# Nutzen der oben angegebenen Modelle durch Angabe des Endpunkts
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceHubEmbeddings(model="https://nomic-text-v15-embedding.llm.mylab.th-luebeck.dev")

# Wir laden ein anderes PDF
loader = PyMuPDFLoader("MNG1_Blatt3_Lösungen.pdf", extract_images=False)
docs = loader.load_and_split(text_splitter)

# Chroma Vectorstore vorbereiten (erst leeren, dann neu anlegen)
# Falls notwendig, löschen des Vektorstores
#    db.delete_collection()
db.delete_collection()
db = Chroma.from_documents(docs, embeddings)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Du bist ein Mathe-Tutor. Beantworte die Frage nur mit dem folgenden Kontext, aber gib nicht die finale Lösung der Aufgabe aus. \
Erkläre es so ausführlich, wie für ein kleines Kind und gib andere Beispiel dafür:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
retriever = db.as_retriever()

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("Wie berechne ich Aufgabe 1b in dem Übungsblatt? ")